In [2]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/NiranJosh101/mlops_basic_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="NiranJosh101"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2b6e0a1d2f111da7d122794576fe6e2cd171cd65"

In [3]:
import os
import sys
from pathlib import Path

# Move up to project root (adjust depending on your folder structure)
PROJECT_ROOT = Path(os.getcwd()).parents[0]  # research → project root
os.chdir(PROJECT_ROOT)
sys.path.append(str(PROJECT_ROOT))

print("Running from:", os.getcwd())

Running from: c:\Users\USER\Desktop\001_MLops_project\001_Datascience_project


In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    mlflow_uri_local: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json #save json added to save metrics

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        
        # read the yaml file stated below
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        # read then extract the dir name from the config file just read
        # this allows us creat the artifacts/ folder (where outputs will be stored)
        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN


        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/NiranJosh101/mlops_basic_project.mlflow",
            mlflow_uri_local="http://127.0.0.1:5000"
        )

        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlflow.models.signature import infer_signature

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config=config

    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mse = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mse, r2

    

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(self.config.target_column, axis=1)
        test_y = test_data[self.config.target_column]

        # mlflow.set_registry_uri("http://localhost:5000")

        # with mlflow.start_run() as run:
        #     predicted_qualities = model.predict(test_x)
        #     rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

        #     print(f"rmse: {rmse}\nmae: {mae}\nr2: {r2}")
        #     scores = {"rmse": rmse, "mae": mae, "r2": r2}
        #     save_json(path=Path(self.config.metric_file_name), data=scores)

        #     mlflow.log_params(self.config.all_params)
        #     mlflow.log_metric("rmse", rmse)
        #     mlflow.log_metric("mae", mae)
        #     mlflow.log_metric("r2", r2)

        #     # ✅ Save model without registering (works with DagsHub)
        #     artifact_path = "model"
        #     mlflow.sklearn.log_model(
        #         sk_model=model,
        #         artifact_path=artifact_path,
        #         input_example=test_x.iloc[:1],
        #         signature=infer_signature(test_x, predicted_qualities),
        #     )

        #     # ✅ Manually register model to local registry
        #     run_id = run.info.run_id
        #     model_uri = f"runs:/{run_id}/{artifact_path}"
        #     mlflow.register_model(
        #         model_uri=model_uri,
        #         name="ElasticnetModel"
        #     )

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment("ElasticNet_Model_Tracking")

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Save locally
            model_path = os.path.join(self.config.root_dir, "model.joblib")
            joblib.dump(model, model_path)

            # Log model manually as artifact
            mlflow.log_artifact(local_path=model_path, artifact_path="model")





    

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-11 17:10:23,303: INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-11 17:10:23,310: INFO: yaml file: params.yaml loaded successfully]
[2025-07-11 17:10:23,324: INFO: yaml file: schema.yaml loaded successfully]
[2025-07-11 17:10:23,329: INFO: create directory at: artifacts]
[2025-07-11 17:10:23,332: INFO: create directory at: artifacts/model_evaluation]
[2025-07-11 17:10:24,074: INFO: json file saved at: artifacts\model_evaluation\metric.json]
🏃 View run unleashed-grouse-805 at: https://dagshub.com/NiranJosh101/mlops_basic_project.mlflow/#/experiments/1/runs/b9f198b5b88049fb97d78db1dd0ab1b4
🧪 View experiment at: https://dagshub.com/NiranJosh101/mlops_basic_project.mlflow/#/experiments/1
